In [61]:
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
import torchvision.transforms as T
from torch import Tensor
from torch.utils.data import DataLoader, random_split

modules_path = 'C:/Users/PMLS/FYP/Modules/'
dataset_path = 'C:/Users/PMLS/FYP/Dataset/STMM'
save_path = "C:/Users/PMLS/FYP/Models"

sys.path.insert(1, modules_path)
from video_dataset import VideoFrameDataset, ImglistToTensor

if False and torch.xpu.is_available():
    device = torch.device("xpu")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cpu


In [70]:
# Define model
class MLE(torch.nn.Module):
    def __init__(self):
        super(MLE, self).__init__()
        print("A")
        self.layer0 = torch.nn.LSTM(2,2)
        print("B")
        self.layer1 = torch.nn.Linear(2,5)
        print("C")

    def forward(self, x):
        out0 = self.layer0(x)
        out1 = self.layer1(out0)
        return out1

torch.set_default_dtype(torch.float32)
model = MLE().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_function = torch.nn.CrossEntropyLoss()

A
B
C


In [74]:
# Define training loop
def train(model, data):
    model.train()
    for i, (x, y) in enumerate(data):
        print(x)
        x = x.to(device)
        y = y.to(device)

        y_hat = model(x)
        loss = loss_function(y_hat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return loss.item()

def test(model, data, label=""):
    correct = 0
    model.eval()
    with torch.no_grad():
        for i, (x,y) in enumerate(data):
            x = x.to(device)
            y = y.to(device)
            
            y_hat = model(x)
            _, y_hat = torch.max(y_hat,1)
            correct += (y_hat==y).sum()
    print(label+"accuracy:", round(float(correct/(i+1)/batch_size), 3))

In [75]:
GESTURE_MAP_STR2NUM = {'down': 0, 'left': 1, 'right': 2, 'up': 3, "none":4}
def get_dataset(N):
    dataset = []
    for i in range(N):
        # res = 64*np.random.rand(21,2)
        # prev_res = 64*np.random.rand(21,2)
        
        # dresults = res-prev_res
        # dX, dY = np.mean(dresults,0)
        dX, dY = np.random.randn(2,1) * 32

        if abs(dX) < epsilon and abs(dY) < epsilon:
            prediction_result = "none"
        elif abs(dX) > abs(dY):
            if dX > 0:
                prediction_result="left"
            else:
                prediction_result="right"
        elif abs(dX) < abs(dY):
            if dY > 0:
                prediction_result="down"
            else:
                prediction_result="up"
        else:
            prediction_result = "none"

        data = torch.from_numpy(np.array([dX[0],dY[0]]).astype(np.float32))
        label = torch.tensor(GESTURE_MAP_STR2NUM[prediction_result])
        dataset.append([data, label])
    return dataset

epsilon = 1
dataset = get_dataset(18400)
batch_size = 16
num_workers = 0
train_dataset, test_dataset = random_split(dataset, [0.5,0.5])
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, num_workers=num_workers, shuffle=True)

In [76]:
%%time

# Train
N=40
for epoch in range(N):
    loss = train(model, train_loader)
print("loss =", loss )

TypeError: linear(): argument 'input' (position 1) must be Tensor, not tuple

In [59]:
test(model, test_loader)

accuracy: 0.994


In [60]:
torch.save(model, "../model/MLE.model")